In [1]:
from ReRank import ReRank
from tira.third_party_integrations import ensure_pyterrier_is_loaded, persist_and_normalize_run
from tira.rest_api_client import Client
import pyterrier as pt
import pandas as pd

# Create a REST client to the TIRA platform for retrieving the pre-indexed data.
ensure_pyterrier_is_loaded()
tira = Client()

PyTerrier 0.10.0 has loaded Terrier 5.8 (built by craigm on 2023-11-01 18:05) and terrier-helper 0.0.8

No etc/terrier.properties, using terrier.default.properties for bootstrap configuration.


In [2]:
# The dataset: the union of the IR Anthology and the ACL Anthology
# This line creates an IRDSDataset object and registers it under the name provided as an argument.
pt_dataset = pt.get_dataset('irds:ir-lab-sose-2024/ir-acl-anthology-20240504-training')

# A (pre-built) PyTerrier index loaded from TIRA
index = tira.pt.index('ir-lab-sose-2024/tira-ir-starter/Index (tira-ir-starter-pyterrier)', pt_dataset)

# Do the baseline retrieval
bm25 = pt.BatchRetrieve(index, wmodel="BM25")
run = bm25(pt_dataset.get_topics('text'))

# Create a df with the text documents
documents = pt_dataset.get_corpus_iter() 
# Extract docno and text into a DataFrame
doc_list = []
for doc in documents:
    doc_list.append(doc)
documents = pd.DataFrame(doc_list)

# Merge the run df and the documents df
run = run.merge(documents, on='docno', how='left')
print('Done. Here are the first 10 entries of the run')
run.head(10)

Download from the Incubator: https://files.webis.de/data-in-production/data-research/tira-zenodo-dump-preparation/ir-lab-sose2024/2024-05-04-16-05-53.zip
	This is only used for last spot checks before archival to Zenodo.


Download: 100%|██████████| 19.5M/19.5M [00:00<00:00, 37.4MiB/s]


Download finished. Extract...
Extraction finished:  /root/.tira/extracted_runs/ir-lab-sose-2024/ir-acl-anthology-20240504-training/tira-ir-starter
Download from the Incubator: https://files.webis.de/data-in-production/data-research/tira-zenodo-dump-preparation/ir-lab-sose2024/ir-acl-anthology-20240504-truth.zip?download=1
	This is only used for last spot checks before archival to Zenodo.


Download: 100%|██████████| 29.6k/29.6k [00:00<00:00, 1.45MiB/s]


Download finished. Extract...
Extraction finished:  /root/.tira/extracted_datasets/ir-lab-sose-2024/ir-acl-anthology-20240504-training/
Download from the Incubator: https://files.webis.de/data-in-production/data-research/tira-zenodo-dump-preparation/ir-lab-sose2024/ir-acl-anthology-20240504-inputs.zip?download=1
	This is only used for last spot checks before archival to Zenodo.


Download: 100%|██████████| 39.4M/39.4M [00:01<00:00, 36.8MiB/s]


Download finished. Extract...
Extraction finished:  /root/.tira/extracted_datasets/ir-lab-sose-2024/ir-acl-anthology-20240504-training/


ir-lab-sose-2024/ir-acl-anthology-20240504-training documents: 100%|██████████| 126958/126958 [00:03<00:00, 33526.83it/s]


Done. Here are the first 10 entries of the run


,qid,docid,docno,rank,score,query,text
0,1,94858,2004.cikm_conference-2004.47,0,15.681777,retrieval system improving effectiveness,A multi-system analysis of document and term s...
1,1,125137,1989.ipm_journal-ir0volumeA25A4.2,1,15.047380,retrieval system improving effectiveness,Improving the effectiveness of retrieval syste...
2,1,125817,2005.ipm_journal-ir0volumeA41A5.11,2,14.144223,retrieval system improving effectiveness,Techniques for improving web retrieval effecti...
3,1,5868,W05-0704,3,14.025748,retrieval system improving effectiveness,Examining the Effect of Improved Context Sensi...
4,1,84876,2016.ntcir_conference-2016.90,4,13.947994,retrieval system improving effectiveness,UB at the NTCIR-12 SpokenQuery&Doc-2: Spoken C...
5,1,82472,1998.sigirconf_conference-98.15,5,13.901647,retrieval system improving effectiveness,Effective Retrieval with Distributed Collectio...
6,1,94415,2008.cikm_conference-2008.183,6,13.808208,retrieval system improving effectiveness,Improve the effectiveness of the opinion retri...
7,1,17496,O01-2005,7,13.749449,retrieval system improving effectiveness,Improving the Effectiveness of Information Ret...
8,1,82490,1998.sigirconf_conference-98.33,8,13.735541,retrieval system improving effectiveness,RELIEF: Combining Expressiveness and Rapidity ...
9,1,124801,2006.ipm_journal-ir0volumeA42A3.2,9,13.569263,retrieval system improving effectiveness,Examining and improving the effectiveness of r...


In [6]:
# Create ReRank object
ReRank_object = ReRank()

# Rerank the documents
run = ReRank_object.rerank_documents(df=run, top_n=5)

# Output the reranked DataFrame
#print(run)

A parameter name that contains `beta` will be renamed internally to `bias`. Please use a different name to suppress this warning.
A parameter name that contains `gamma` will be renamed internally to `weight`. Please use a different name to suppress this warning.
A parameter name that contains `beta` will be renamed internally to `bias`. Please use a different name to suppress this warning.
A parameter name that contains `gamma` will be renamed internally to `weight`. Please use a different name to suppress this warning.
A parameter name that contains `beta` will be renamed internally to `bias`. Please use a different name to suppress this warning.
A parameter name that contains `gamma` will be renamed internally to `weight`. Please use a different name to suppress this warning.
A parameter name that contains `beta` will be renamed internally to `bias`. Please use a different name to suppress this warning.
A parameter name that contains `gamma` will be renamed internally to `weight`. Pl

In [ ]:
# Evaluate the results
qrels_df = pt_dataset.get_qrels()
eval = pt.Evaluate(run, qrels_df, metrics=["map", "ndcg", "ndcg_cut.10", "recip_rank", "recall_100"])
print("Evaluation Metrics:")
print(eval)

In [ ]:
# Filter run to include only judged documents
qrels_df = pt_dataset.get_qrels()
judged_docnos = qrels_df['docno'].unique()
filtered_run = run[run['docno'].isin(judged_docnos)]

# Evaluate the results
eval = pt.Evaluate(filtered_run, qrels_df, metrics=["map", "ndcg", "ndcg_cut.10", "recip_rank", "recall_100"])
print("Evaluation Metrics:")
print(eval)

In [ ]:
# Persist the run file for subsequent evaluations
persist_and_normalize_run(run, system_name='bm25-ReRank', default_output='../runs')

### Compare to BM25 Baseline

In [ ]:
# Baseline without Doc2Query
# A (pre-built) PyTerrier index loaded from TIRA
index = tira.pt.index('ir-lab-sose-2024/tira-ir-starter/Index (tira-ir-starter-pyterrier)', pt_dataset)
# Retrieve documents using BM25
bm25 = pt.BatchRetrieve(index, wmodel="BM25")
# Perform retrieval
run = bm25(pt_dataset.get_topics('text'))
# Evaluate the results
eval = pt.Evaluate(run, qrels_df, metrics=["map", "ndcg", "ndcg_cut.10", "recip_rank", "recall_100"])
print("Evaluation Metrics:")
print(eval)

In [ ]:
# Filter run to include only judged documents
qrels_df = pt_dataset.get_qrels()
judged_docnos = qrels_df['docno'].unique()
filtered_run = run[run['docno'].isin(judged_docnos)]

# Evaluate the results
eval = pt.Evaluate(filtered_run, qrels_df, metrics=["map", "ndcg", "ndcg_cut.10", "recip_rank", "recall_100"])
print("Evaluation Metrics:")
print(eval)